In [3]:
import pandas as pd
import re
import io
import os
import glob
import time
from unicodedata import normalize
from sqlalchemy import create_engine
import psycopg2 as ps
from sqlalchemy import text

#database information to connect
host = 'ds4ateam77.cxdzzcrtbiby.us-east-2.rds.amazonaws.com'
port = 5432
user = 'postgres'
password = '12345678'
database = 'teate'
def runQuery(sql):
    engine=create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}',max_overflow=30)
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

pd.set_option('display.max_columns', None)


In [32]:
start_time = time.time()
SQL_Query = runQuery("""

with MaxDate as (
    select 
        max(fechapedido::date) Date
    from fact_orders_categorized

), sub as (

    select 
          categoria, subcategoria, nombrefabricante
        , sum(valortotalpedido) as Valor
        , round(sum(valortotalpedido*1.00)/
            (select sum(valortotalpedido) 
            from fact_orders_categorized
            where fechapedido::date > (select Date from MaxDate) - INTERVAL '90 day'
            ),2) as Pareto
    from fact_orders_categorized
    where fechapedido::date > (select Date from MaxDate) - INTERVAL '90 day'
    group by 1,2,3
    order by 4 desc
), paretoacum as (

select 
row_number() over(order by Valor desc) rank
, *
, sum(Pareto) over (order by Valor desc rows between unbounded preceding and current row) as pareto_acum

from sub 

), HIPERCATGORIES AS ( 

select * 
, case when pareto_acum < 0.80 then 
         replace(replace(concat(categoria,'_', subcategoria,'_', nombrefabricante),' ','_'),'','_') 
    else 
        replace(replace(concat(categoria,'_otros'),' ','_'),'','_') end as Hipercategoria_MASK
, replace(replace(concat(categoria,'_', subcategoria,'_', nombrefabricante),' ','_'),'','_') AS Hipercategoria
from paretoacum
)

select 
  to_char(date_trunc('week',fechapedido::date),'YYYY-MM-DD') as Date
, Coalesce(H.Hipercategoria_MASK,'Otros') linea_de_abastecimiento
, SUM(valortotalpedido) VALOR
from fact_orders_categorized F
LEFT JOIN HIPERCATGORIES H
    ON replace(replace(concat(F.categoria,'_', F.subcategoria,'_', F.nombrefabricante),' ','_'),'','_')
        = H.Hipercategoria
group by 1,2
order by 1,2 desc

""") 
print(str(len(SQL_Query))+' rows')
print("--- %s seconds ---" % (time.time() - start_time))
DATA=SQL_Query
DATA=DATA.pivot(index='date', columns='linea_de_abastecimiento', values='valor')

DATA_not_nulls=DATA.fillna(method='ffill')
DATA_not_nulls=DATA_not_nulls.fillna(method='bfill')
DATA_not_nulls
DATA=DATA_not_nulls
DATA

2156 rows
--- 14.163713455200195 seconds ---


In [33]:
#connect drop table if exists and create it 
data=DATA.reset_index()
TABLE_NAME = 'vec_input'
connDB = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = connDB.raw_connection()
cursor = conn.cursor()
data.head(0).to_sql(TABLE_NAME, connDB, if_exists='replace',index=False)
conn.commit()
conn.close()

#Upload Data in cycles
S_=0
D_=1000
start_time = time.time()
print("Start at 0 seconds ---")
for n in range(S_,len(data),D_):
    S_=n
    E_=n+D_
    df2=data[(data.index>=S_) & (data.index<E_)]
    #connect to db
    conn = connDB.raw_connection()
    cursor = conn.cursor()
    engine=connDB
    conn = engine.raw_connection()
    cur = conn.cursor()
    output = io.StringIO()
    df2.to_csv(output, sep='\t', header=False, index=False)
    output.seek(0)
    cur.copy_from(output, TABLE_NAME, null="" , sep='\t') # null values become ''
    conn.commit()
    Comp = round((n+D_)/len(data)*100,1)
    Records = str(n+D_)
    print(Records+" "+str(min(Comp,100))+" pct--- %s seconds" % round((time.time() - start_time),4))
df2=data[(data.index>=E_) & (data.index<=len(data))]
print('Done 100%')
print("--- %s seconds ---" % (time.time() - start_time))
conn.close()


Start at 0 seconds ---
1000 100 pct--- 0.7239 seconds
Done 100%
--- 0.7261033058166504 seconds ---


In [34]:
start_time = time.time()
SQL_Query = runQuery("""

select * from vec_input

""") 
print(str(len(SQL_Query))+' rows')
print("--- %s seconds ---" % (time.time() - start_time))
SQL_Query


88 rows
--- 2.818427562713623 seconds ---


,date,ALIMENTOS_ACEITES_Y_VINAGRES_TEAM_FOODS_COLOMBIA_S.A.,"ALIMENTOS_ARROZ,_GRANOS_Y_PASTA_AGROINDUSTRIAL_MOLINO_SONORA_A.","ALIMENTOS_ARROZ,_GRANOS_Y_PASTA_ARROCERA_LA_ESMERALDA_S.A.S","ALIMENTOS_ARROZ,_GRANOS_Y_PASTA_ARROZ_FLORHUILA_S.A","ALIMENTOS_ARROZ,_GRANOS_Y_PASTA_ORGANIZACION_ROA_S.A","ALIMENTOS_AZUCAR,_PANELA_Y_ENDULZANTES_EMPAQUETADOS_EL_TRECE_S.","ALIMENTOS_AZUCAR,_PANELA_Y_ENDULZANTES_MAYAG_EZ_S.A","ALIMENTOS_AZUCAR,_PANELA_Y_ENDULZANTES_PRODUCTORA_Y_COMERCIALIZ","ALIMENTOS_CONDIMENTOS,_CALDOS_Y_SAL_REFISAL","ALIMENTOS_ENLATADOS,_SALSAS_Y_CONSERVAS_COMPAIA_NACIONAL_DE_L",ALIMENTOS_otros,ASEO_DEL_HOGAR_otros,ASEO_HOGAR_JABON_EN_BARRA_DETERGENTES_LTDA.,ASEO_HOGAR_otros,ASEO_PERSONAL_CREMA_DENTAL_COLGATE_PALMOLIVE_COMPAIA,ASEO_PERSONAL_OTROS_ASEO_PERSONAL_EMPAQUETADOS_EL_TRECE_S.A.S,ASEO_PERSONAL_PAPEL_HIGIENICO_DRYPERS_ANDINA_S.A,ASEO_PERSONAL_otros,BEBIDAS_CAF_MOLIDO_COMERCIALIZADORA_PROCON_S.A,BEBIDAS_GASEOSAS_INDUSTRIA_NACIONAL_DE_GASEOSAS_S.A.,BEBIDAS_otros,CONFITERIA_PASABOCAS_PRODUCTOS_YUPI_S.A.S,CONFITERIA_Y_SNACKS_CEREALES_KELLOGG_DE_COLOMBIA_S.A,CONFITERIA_Y_SNACKS_otros,CONFITERIA_otros,DETERGENTE_PARA_ROPA_otros,MASCOTAS_otros,MEDICAMENTOS_otros,MUNDO_BEBES_otros,MUNDO_BEBE_otros,OTRAS_CATEGORIAS_otros,Otros,SALUD_Y_BIENESTAR_otros,_BEBIDAS_otros
0,2018-12-31,271400.0,7086200.0,5274250.0,3182800.0,4172900.0,65000.0,2334100.0,20000.0,497400.0,9312000.0,2854450.0,56100.0,1528200.0,5792800.0,169400.0,31500.0,2338750.0,874150.0,11654050.0,745809.0,697650.0,489350.0,954650.0,83600.0,1161900.0,142100.0,906700.0,113400.0,339000.0,339000.0,71400.0,1601839.0,4500.0,21600.0
1,2019-01-07,507700.0,7086200.0,5319250.0,5869200.0,8757600.0,65000.0,1979800.0,20000.0,497400.0,9312000.0,5425600.0,22950.0,794700.0,2181250.0,61600.0,31500.0,1548500.0,915900.0,11654050.0,745809.0,697650.0,489350.0,954650.0,65950.0,1450550.0,142100.0,906700.0,113400.0,339000.0,339000.0,71400.0,1654452.0,198500.0,21600.0
2,2019-01-14,361500.0,7086200.0,6952250.0,5314400.0,6609150.0,65000.0,2095400.0,20000.0,497400.0,9312000.0,4490250.0,35700.0,815100.0,2507150.0,46200.0,31500.0,5843250.0,842750.0,11654050.0,745809.0,697650.0,489350.0,954650.0,63200.0,935550.0,142100.0,906700.0,113400.0,339000.0,339000.0,71400.0,8852506.0,13500.0,21600.0
3,2019-01-21,163650.0,7086200.0,3152000.0,5375300.0,2341000.0,65000.0,1623400.0,20000.0,497400.0,9312000.0,2614550.0,15300.0,927450.0,2178950.0,46200.0,31500.0,1257250.0,793450.0,11654050.0,745809.0,697650.0,489350.0,954650.0,104600.0,2106800.0,142100.0,906700.0,113400.0,339000.0,339000.0,71400.0,13139291.0,59400.0,21600.0
4,2019-01-28,349450.0,7086200.0,5824250.0,4542600.0,3291300.0,65000.0,2893400.0,20000.0,497400.0,9312000.0,4844000.0,76500.0,3020550.0,7468300.0,46200.0,31500.0,2393750.0,1450150.0,11654050.0,745809.0,697650.0,489350.0,954650.0,88550.0,855050.0,142100.0,906700.0,113400.0,339000.0,339000.0,71400.0,9574541.0,65700.0,21600.0
5,2019-02-04,219500.0,7086200.0,12044750.0,2999200.0,7819900.0,65000.0,3128800.0,20000.0,497400.0,9312000.0,4398850.0,30600.0,1303250.0,3823400.0,46200.0,31500.0,4111875.0,1057600.0,11654050.0,745809.0,697650.0,489350.0,954650.0,59750.0,1950350.0,142100.0,906700.0,113400.0,339000.0,339000.0,71400.0,11090054.0,113400.0,21600.0
6,2019-02-11,610000.0,7086200.0,11513750.0,6296400.0,7731900.0,65000.0,3956000.0,20000.0,497400.0,9312000.0,5955200.0,27200.0,1602150.0,4991800.0,46200.0,31500.0,1815500.0,1559000.0,11654050.0,745809.0,697650.0,489350.0,954650.0,122450.0,1122300.0,142100.0,906700.0,113400.0,339000.0,339000.0,71400.0,14782739.0,240300.0,21600.0
7,2019-02-18,418000.0,7086200.0,16523000.0,8405100.0,12181400.0,65000.0,2835000.0,20000.0,638000.0,9312000.0,4527200.0,60350.0,2187950.0,5227400.0,46200.0,31500.0,1934625.0,1088150.0,11654050.0,745809.0,697650.0,489350.0,954650.0,93550.0,1049950.0,142100.0,906700.0,113400.0,339000.0,339000.0,71400.0,17996102.0,52200.0,21600.0
8,2019-02-25,331100.0,7086200.0,8484750.0,4572300.0,6317900.0,65000.0,7116700.0,20000.0,479850.0,93120

In [8]:
data.columns()

TypeError: 'Index' object is not callable